#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2019


# Homework 2:   word2vec + SVM + Evaluation

### 100 points [6% of your final grade]

### Due: Tuesday, February 26, 2019 by 11:59pm

*Goals of this homework:* Understand word2vec-like term embeddings,  explore real-world challenges with SVM-based classifiers, understand and implement several evaluation metrics.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw2.ipynb`. For example, my homework submission would be something like `555001234_hw2.ipynb`. Submit this notebook via eCampus (look for the homework 2 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1: Term embeddings + SVM (80 points)

### Dataset


For this homework, we will still play with Yelp reviews from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge). As in Homework 1, you'll see that each line corresponds to a review on a particular business. Each review has a unique "ID" and the text content is in the "review" field. Additionally, this time, we also offer you the "label". If `label=1`, it means that this review is `Food-relevant`. If `label=0`, it means that this review is `Food-irrelevant`. Similarly, we have already done some basic preprocessing on the reviews, so you can just tokenize each review using whitespace.

There are about 40,000 reviews in total, in which about 20,000 reviews are "Food-irrelevant". We split the review data into two sets. *review_train.json* is the training set. *review_test.json* is the testing set. 

In [2]:
# Please load the dataset
# Your code below


train_relevant={}
train_irrelevant={}
test_relevant={}
test_irrelevant={}

train_len=0
import json
with open('review_train.json') as f:
    for line in f:
        train_len += 1
        data = json.loads(line)
        review_data = data['review'].split()
        id = data['id']
        label = data['label']
        if(label == 1):
            train_relevant[id] = review_data
        else:
            train_irrelevant[id] = review_data

test_len = 0
with open('review_test.json') as f:
    for line in f:
        test_len += 1
        data = json.loads(line)
        review_data = data['review'].split()
        id = data['id']
        label = data['label']
        if(label == 1):
            test_relevant[id] = review_data
        else:
            test_irrelevant[id] = review_data
print('train_len', train_len)
print(len(train_relevant), len(train_irrelevant))
print('test_len', test_len)
print(len(test_relevant), len(test_irrelevant))

train_len 27813
13938 13875
test_len 11920
5945 5975


###  Pre-trained term embeddings

To save your time, you can make use of  pre-trained term embeddings. In this homework, we are using one of the great pre-trained models from [GloVe](https://nlp.stanford.edu/projects/glove/) based on 2 billion tweets. GloVe is quite similar to word2vec. Unzip the *glove.6B.50d.txt.zip* file and run the code below. You will be able to load the term embeddings model, with which each word can be represented with a 50-dimension vector.

In [3]:
# reload the pre-trained term embeddings
import numpy as np

with open("glove.6B.50d.txt", "rb") as lines:
    #model = {line.split()[0]: np.array(map(float, line.split()[1:]))
    #       for line in lines}
    model={}
    for line in lines:
        split_w = line.split()
        key = split_w[0]
        val = [float(i) for i in split_w[1:]]
        model[key] = np.array(val)
    

Now, you have a vector representation for each word. First, we use the simple (arithmetic) **mean** of these vectors of words in a review to represent the review. *Note: Just ignore those words which are not in the corpus of this pre-trained model.*

In [4]:
# Please figure out the vector representation for each review in the training data and testing data.
# Your code below
import numpy as np
i = 0

def GetMeans(dataset):
    h = {}
    for key,words in dataset.items():
        vec = np.zeros(50,)
        for i in range(0,len(words)):
            word = bytes(bytearray(words[i], 'utf-8'))
            if word not in model:
                continue
            vec = np.add(vec , model[word])
        #h[key] = np.mean(vec)
        h[key] = [x/len(words) for x in vec]
    return h


relevant_means = GetMeans(train_relevant)
irrelevant_means = GetMeans(train_irrelevant)

def AppendMeans(m, val, mean_vals, Y_vals):  
    for k,v in m.items():
        mean_vals.append(v)
        Y_vals.append(val)
    return [mean_vals,Y_vals]

[a,b] = AppendMeans(relevant_means, 1, [], [])
[train_means,train_Y] = AppendMeans(irrelevant_means, 0, a, b)

#print(train_means[:3])
train_means=np.array(train_means)
print(train_means.shape)
print(len(train_Y))


(27813, 50)
27813


### SVM

With the vector representations you get for each review, please train an SVM model to predict whether a given review is food-relevant or not. **You do not need to implement any classifier from scratch. You may use scikit-learn's built-in capabilities.** You can only train your model with reviews in *review_train.json*.

In [5]:
# SVM model training
# Your code here

from sklearn import svm
clf = svm.SVC(gamma='scale', kernel = 'rbf')
print(clf.fit(train_means, train_Y))
#clf.score(np.array(train_means), train_Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


Your goal is to predict whether a given review is food-relevant or not. Please report the overall accuracy, precision and recall of your model on the **testing data**. You should **implement the functions for accuracy, precision, and recall**.

In [6]:
# Your code here

[c,d] = AppendMeans(GetMeans(test_irrelevant), 0, [], [])
[test_means,test_Y] = AppendMeans(GetMeans(test_relevant), 1, c, d)

predicted = np.array(clf.predict(test_means))

diff = [np.abs(a-b) for a,b in zip(test_Y, predicted)]

accuracy = 1-(diff.count(1)/len(predicted))

actual = test_Y

fneg = 0
fpos = 0
tpos =0
for i in range(len(actual)):
    if(actual[i]-predicted[i] == 1):
        fneg+=1
    if(predicted[i]-actual[i] == 1):
        fpos+=1
    if(predicted[i]+actual[i] == 2):
        tpos+=1
print('Pre-trained term embeddings:')
print('Accuracy', accuracy)
P= tpos/(tpos+fpos)
R= tpos/(tpos+fneg)
print('Precision', P)
print('Recall', R)

Pre-trained term embeddings:
Accuracy 0.9034395973154362
Precision 0.87999365884591
Recall 0.9337258200168208


### Document-based embeddings

Instead of taking the mean of term embeddings, you can directly train a **doc2vec** model for paragraph or document embeddings. You can refer to the paper [Distributed Representations of Sentences and Documents](https://arxiv.org/pdf/1405.4053v2.pdf) for more details. And in this homework, you can make use of the implementation in [gensim](https://radimrehurek.com/gensim/models/doc2vec.html).

Now, you need to:
* Train a doc2vec model based on all reviews you have (training + testing sets).
* Use the embeddings from your doc2vec model to represent each review and train a new SVM model.
* Report the overall accuracy, precision and recall of your model on the testing data.

In [7]:
# Train a doc2vec
# Your code here
import gensim
import os
import collections
import smart_open
import random



def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            #print(line)
            data = json.loads(line)
            review_data = data['review']
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(review_data), [i])
            '''    
            if tokens_only:
                yield gensim.utils.simple_preprocess(review_data)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(review_data), [i])
            '''
train_corpus = list(read_corpus('review_train.json'))
test_corpus = list(read_corpus('review_test.json', tokens_only=True))
epochs_count = 100
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, epochs=100)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)



C:\Users\kavya\Anaconda2\envs\cs670\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 4min 45s


In [8]:
# Train a SVM
# Your code here
def GetWords(dataset):
    h = {}
    for key,words in dataset.items():
        vec = np.zeros(50,)
        vec = np.add(vec, model.infer_vector(words))
        h[key] = vec
    return h

def Append(m, val, mean_vals, Y_vals):  
    for k,v in m.items():
        mean_vals.append(v)
        Y_vals.append(val)
    return [mean_vals,Y_vals]

[a,b] = Append(GetWords(train_relevant), 1, [], [])
[train_vecs,train_vec_Y] = Append(GetWords(train_irrelevant), 0, a, b)

from sklearn import svm
clf = svm.SVC(gamma='scale', kernel='rbf')
print('training SVM')
print(clf.fit(train_vecs, train_vec_Y))



training SVM
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [13]:
# Report the performance
# Your code here

[c1,d1] = Append(GetWords(test_irrelevant), 0, [], [])
[test_vecs,test_vec_Y] = Append(GetWords(test_relevant), 1, c1, d1)

predicted = np.array(clf.predict(test_vecs))

diff = [np.abs(a-b) for a,b in zip(test_vec_Y, predicted)]

accuracy = 1-(diff.count(1)/len(predicted))

actual = test_vec_Y

fneg = 0
fpos = 0
tpos =0
for i in range(len(actual)):
    if(actual[i]-predicted[i] == 1):
        fneg+=1
    if(predicted[i]-actual[i] == 1):
        fpos+=1
    if(predicted[i]+actual[i] == 2):
        tpos+=1
print('Gensim Doc2Vec: epochs =', epochs_count)
print('Accuracy', accuracy)
P= tpos/(tpos+fpos)
R= tpos/(tpos+fneg)
print('Precision', P)
print('Recall', R)

Gensim Doc2Vec: epochs = 100
Accuracy 0.9046979865771811
Precision 0.8975037196230782
Recall 0.9132043734230446


What do you observe? How different are your results for the term-based average approach vs. the doc2vec approach? Why do you think this is?

Pre-trained term embeddings:
Accuracy 0.9034395973154362
Precision 0.87999365884591
Recall 0.9337258200168208

Gensim Doc2Vec: epochs = 100
Accuracy 0.9046979865771811
Precision 0.8975037196230782
Recall 0.9132043734230446

From the above results, I observed that Doc2Vec is more accurate and precise than pre-trained term embeddings. I believe this is because of the 50 dimensional weights being decided by using the train and test data itself in Doc2Vec. 

On the other hand, recall value is higher for pre-trained term embeddings. Recall value tells how accurately we can predict the positives. I believe since the pre-term embeddings give an original distribution without getting a glimpse of the train or the test data, it would predict the positives better than word2vec where the weights are biased based on the train&test data.

### Can you do better?

Finally, see if you can do better than either the word- or doc- based embeddings approach for classification. You may explore new features, new classifiers, etc. Whatever you like. Just provide your code and a justification.

In [1]:
# your code here

print('training the model')
'''
X = train_vecs
y = train_vec_Y
predict_X = test_vecs
actual = test_vec_Y
'''
X = train_means
y = train_Y
predict_X = test_means
actual = test_Y

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def GetAccuracy(classifier):
    print('---------')
    #print(
    print(classifier.fit(X,y))
    predicted = np.array(classifier.predict(predict_X))

    diff = [np.abs(a-b) for a,b in zip(actual, predicted)]
    accuracy = 1-(diff.count(1)/len(predicted))

    fneg = 0
    fpos = 0
    tpos =0
    for i in range(len(actual)):
        if(actual[i]-predicted[i] == 1):
            fneg+=1
        if(predicted[i]-actual[i] == 1):
            fpos+=1
        if(predicted[i]+actual[i] == 2):
            tpos+=1
    print('Accuracy', accuracy)
    
    P= tpos/(tpos+fpos)
    R= tpos/(tpos+fneg)
    print('Precision', P)
    print('Recall', R)
    print('---------')

'''
clf = KNeighborsClassifier(n_neighbors=5)
GetAccuracy(clf)
clf = QuadraticDiscriminantAnalysis()
GetAccuracy(clf)
clf = DecisionTreeClassifier(random_state=0)
GetAccuracy(clf)
clf = RandomForestClassifier()
GetAccuracy(clf)
clf = AdaBoostClassifier()
GetAccuracy(clf)
clf = GradientBoostingClassifier()
GetAccuracy(clf)
clf = GaussianNB()
GetAccuracy(clf)
clf = SVC(kernel="rbf")
GetAccuracy(clf)
clf = NuSVC(probability=True)
GetAccuracy(clf)
'''
clf = LinearDiscriminantAnalysis()
GetAccuracy(clf)



training the model


NameError: name 'train_vecs' is not defined

In [62]:
print('After trying out with various available classifiers and measuring their accuracy. I realized that the closest accuracy I could get to the previous results is of LinearDiscriminantAnalysis with accuracy = 0.8926174496644296')
print()
print('I believe tweaking the parameters of LDA would help get better accuracy.')

After trying out with various available classifiers and measuring their accuracy. I realized that the closest accuracy I could get to the previous results is of LinearDiscriminantAnalysis with accuracy = 0.8926174496644296

I believe tweaking the parameters of LDA would help get better accuracy.


# Part 2: NDCG (20 points)

You calculated the recall and precision in Part 1 and now you get a chance to implement NDCG. 

Assume that Amy searches for "food-relevant" reviews in the **testing set** on two search engines `A` and `B`. Since the ground-truth labels for the reviews are unknown to A and B, they need to make a prediction for each review and then return a ranked list of results based on their probabilities. The results from A are in *search_result_A.json*, and the results from B are in *search_result_B.json*. Each line contains the id of a review and its corresponding ranking.

You can check their labels in *review_test.json* while calculating the NDCG scores. If a review is "food-relevant", the relevance score is 1. Otherwise, the relevance score is 0.

In [25]:
# NDCG for search_result_A.json
# Your code here
from operator import itemgetter

def GetNdcg(filename):
    my_list = []
    with open(filename) as f:
        dcg=0
        for line in f:
            data = json.loads(line)
            review_id = data['id']
            i = data['rank']
            label=0
            if review_id in test_relevant:
                label=1
            my_list.append([review_id, label])
            dcg += label/np.log2(1+i)

    ideal_list = sorted(my_list, key=itemgetter(1), reverse=True)
    idcg=0
    for j in range(len(ideal_list)):
        [review_id, label] = ideal_list[j]
        i = j+1
        idcg += label/np.log2(1+i)
    ndcg = dcg/idcg
    
    return ndcg

print('ndcg for search result A:', GetNdcg('search_result_A.json'))

ndcg for search result A: 0.96916958684041


In [26]:
# NDCG for search_result_B.json
# Your code here
print('ndcg for search result B:', GetNdcg('search_result_B.json'))



ndcg for search result B: 0.9972073294831962


## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*

In [ ]:
I followed doc2vec tutorial from here:
    https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb